In [2]:
from datasets import load_dataset
dataset = load_dataset("jerteh/SrpKorNews")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 337
    })
})

In [4]:
print(dataset['train']['text'][0][:100])
print(type(dataset['train']['text']))
print(len(dataset['train']['text']))

<s>Mečka na usijanom limenom krovu</s><s>Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil
<class 'list'>
337


In [5]:
from pprint import pprint
pprint(dataset['train']['text'][0][:1000])

('<s>Mečka na usijanom limenom krovu</s><s>Piše: Dušan Vidaković Kada je '
 'austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, '
 'odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 '
 'hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime '
 'njegove ćerke.</s><s>Ljudi iz kanstatske firme, naravno, nisu imali ništa '
 'protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici '
 'kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku '
 'konkvistadora znaci ljupkost, umilnost, prijatnost.</s><s>Jelinekova '
 'porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je '
 'preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do '
 'danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile '
 'svojstvenih vozačima koji ne kriju brige u novčaniku.</s><s>Međutim, upravo '
 'legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu

In [6]:
texts = dataset['train']['text'][:100]
pprint(texts[2][:500])

('<s>Iskoristite poslednju šansu</s><s>Dok šefovi država sveta u Meksiku '
 'uzalud troše četiri dragocena dana pokušavajući da ubede javnost da '
 'saopštenjima za novinare mogu da reše svetske ekonomske probleme, i probleme '
 'životne sredine u Brazilu, evropski lideri zaista imaju poslednju šansu da '
 'odluče o svojoj sudbini.</s><s>Poslednju šansu, jer ako ne urade ništa, '
 'opstanak evra će ponovo, narednog leta, zavisiti od masivnih intervencija '
 'centralnih banaka.</s><s>A sve naše zemlje ubrzo će preplaviti ')


In [7]:
# # Prepare documents
# doc_chunks = [text for i in range(0, len(texts), 1000) for text in texts[i:i+1000]]
# print(doc_chunks[0])
# print(len(doc_chunks[0]))

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer_jerteh = AutoTokenizer.from_pretrained("jerteh/Jerteh-355")
model_jerteh = AutoModelForMaskedLM.from_pretrained("jerteh/Jerteh-355")

In [9]:
embeddings = tokenizer_jerteh(texts)
print(embeddings[0])
print(len(embeddings[0]))

Encoding(num_tokens=88470, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
88470


In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [11]:
# from bertopic import BERTopic
# from umap import UMAP

# # Prepare embeddings
# embeddings = tokenizer(texts)

# # Train BERTopic
# topic_model = BERTopic().fit(texts, embeddings)

# # Run the visualization with the original embeddings
# topic_model.visualize_documents(texts, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(texts, reduced_embeddings=reduced_embeddings)
#embeddings

In [12]:
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import LongTensor, no_grad
from scipy import spatial
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer# Defining model pipeline

umap_model = UMAP(n_neighbors= 2, n_components= 2, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size = 2, min_samples = 4, metric ='euclidean', cluster_selection_method = 'eom', prediction_data=True)
vectorizer_model = CountVectorizer(min_df = 2, max_df = 0.8, ngram_range = (1, 2))

topic_model = BERTopic(

  # Pipeline models
  
  embedding_model=model_jerteh,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  min_topic_size = 3,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

: 

In [9]:
# Train model
topics, probs = topic_model.fit_transform(texts)

# Show topics
topics_bert = topic_model.get_topic_info()
topics_bert

2024-08-13 15:38:01,753 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-08-13 15:40:00,613 - BERTopic - Embedding - Completed ✓
2024-08-13 15:40:00,614 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-13 15:40:06,710 - BERTopic - Dimensionality - Completed ✓
2024-08-13 15:40:06,711 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-13 15:40:06,717 - BERTopic - Cluster - Completed ✓
2024-08-13 15:40:06,719 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-13 15:42:02,829 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,8,-1_ilirika_szs_prvorođeni_bojkota,"[ilirika, szs, prvorođeni, bojkota, mhe, preds...",[<s>Rekonstruisano 200 benzinskih stanica</s><...
1,0,34,0_font_font family_size_font size,"[font, font family, size, font size, 12pt, siz...",[<s>Plavi orkestar i Bajaga večeras ispred Dom...
2,1,33,1_font_večeras od_font family_size,"[font, večeras od, font family, size, font siz...",[<s>Pozdrav levici</s><s>Janis Varufakis rado ...
3,2,10,2_belexline_font_srx_indeksnih,"[belexline, font, srx, indeksnih, trgovano, fo...",[<s>Dinarski krediti Laiki banke bez valutne k...
4,3,10,3_font_font family_font size_size,"[font, font family, font size, size, vode će, ...",[<s>Centralni trgovi bez prevoza do pet izjutr...
5,4,5,4_renault_nm_nissan_baus,"[renault, nm, nissan, baus, najboljiauto, najb...",[<s>„Nemci“ do 2.000 evra najtraženiji polovnj...


In [13]:
topcs_jerteh, probs_jerteh = topic_model.fit_transform(texts)
# Show topics
topics_jerteh = topic_model.get_topic_info()
topics_jerteh

2024-08-13 15:55:52,116 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [51]:
new_topics = new_topics = topic_model.reduce_outliers(texts, topics, strategy="embeddings")
topic_model.update_topics(texts, topics=new_topics)
topic_model.get_topic_info()

2024-08-13 10:37:19,831 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,9,0_je_da_na_se,"[je, da, na, se, za, su, od, sa, ne, će]",[<s>Pravi med je skup:</s><s>Evo kakve namete ...
1,1,11,1_je_da_se_na,"[je, da, se, na, za, su, od, sa, to, koji]",[<s>Prvi srpski ustanak:</s><s>Kad je narod kr...
